- Needed Packages

In [ ]:
!pip install torch matplotlib numpy

In [ ]:
# Installing packages for optim
# Muon
!git clone https://github.com/KellerJordan/Muon.git
!pip install -e Muon
# SOAP
!git clone https://github.com/nikhilvyas/SOAP.git
# Shampoo
!git config core.sparseCheckout true
!git clone --filter=blob:none --no-checkout https://github.com/google-research/google-research/  google_research
%cd google_research/
!git sparse-checkout set --cone
!git checkout master
!git sparse-checkout set scalable_shampoo
!pip install -r scalable_shampoo/requirements.txt
%cd ..

# Pleas check the repo before
import os, sys
sys.path.append(os.path.join(os.getcwd(),"google_research/scalable_shampoo/pytorch"))

- Imported Packages

In [ ]:
import math
import torch
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from google_research.scalable_shampoo.pytorch.shampoo import Shampoo
from muon import Muon
from SOAP.soap import SOAP

from typing import Dict, Callable,Union


- Handle distributed optimizers

In [ ]:
# for distributed Muon (this is on one GPU)
import torch.distributed as dist

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
backend = 'nccl' if torch.cuda.is_available() else 'gloo'


# Set necessary environment variables
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "29500"

# Initialize the process group
if not dist.is_initialized():
    dist.init_process_group(backend=backend, init_method="env://")




-  The Rosenbrock Function and Its Non-Convexity


## Introduction
The **Rosenbrock function**, also known as the **banana function**, is a common test problem for optimization algorithms. It is widely used to assess the performance of numerical optimization methods due to its challenging non-convex landscape.

## Definition
The Rosenbrock function in two dimensions is defined as:

$$ f(x, y) = (a - x)^2 + b(y - x^2)^2 $$

where typically **a = 1** and **b = 100**.

The function can be generalized to higher dimensions as (we take n=2):

$$ f(\mathbf{x}) = \sum_{i=1}^{n-1} \, [a - x_i]^2 + b(x_{i+1} - x_i^2)^2 $$

## Properties
- **Global Minimum:** At \( x^* = (1,1) \), where \( f(x^*) = 0 \).
- **Non-Convexity:** The function has a narrow, curved valley leading to the minimum, making gradient-based optimization difficult.
- **Plateaus & Steep Slopes:** Optimization algorithms struggle due to flat regions and steep inclines.


## Optimization Challenge
- The **narrow valley** leads to slow convergence in gradient-based methods like **gradient descent**.
- **Newton’s method** may perform better but requires second-order derivatives.
- **Evolutionary algorithms** (e.g., Genetic Algorithms, CMA-ES) can be more effective in escaping local traps.


In [ ]:

def rosenbrock(x: float, y: float, a: float = 1, b: float = 100) -> float:
    """
    Computes the Rosenbrock function value for given x and y.
    
    Parameters:
        x (float): The x-coordinate.
        y (float): The y-coordinate.
        a (float, optional): Constant parameter, default is 1.
        b (float, optional): Constant parameter, default is 100.
    
    Returns:
        float: The computed Rosenbrock function value.
    """
    return (a - x) ** 2 + b * (y - x**2) ** 2

def torch_rosenbrock(params: Union[torch.Tensor, list], a: float = 1, b: float = 100) -> torch.Tensor:
    """
    Computes the Rosenbrock function value for given parameters in a PyTorch-compatible form.
    
    Parameters:
        params (Union[torch.Tensor, list]): A tensor or list containing x and y values.
        a (float, optional): Constant parameter, default is 1.
        b (float, optional): Constant parameter, default is 100.
    
    Returns:
        torch.Tensor: The computed Rosenbrock function value as a PyTorch tensor.
    """
    try:
        x, y = params
        return (a - x) ** 2 + b * (y - x**2) ** 2
    except Exception as e:
        raise ValueError(f"Invalid input for torch_rosenbrock: {e}")


- Visualization in 2D 

In [ ]:

from mpl_toolkits.mplot3d import Axes3D

def rosenbrock(x, y, a=1, b=100):
    return (a - x)**2 + b * (y - x**2)**2

x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x, y)
Z = rosenbrock(X, Y)

plt.figure(figsize=(8,6))
plt.contourf(X, Y, Z, levels=50, cmap='plasma',vmin=-1)
plt.colorbar(label='Function Value')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Contour Plot of the Rosenbrock Function')
plt.show()

- Import Ademamix code

In [ ]:
##### Code from repo https://github.com/nanowell/AdEMAMix-Optimizer-Pytorch/blob/main/AdEMAMix.py

class AdEMAMix(optim.Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999, 0.9999), eps=1e-8,
                 weight_decay=0, alpha=5.0, T_alpha_beta3=None):
        if not 0.0 <= lr:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon value: {eps}")
        assert len(betas) == 3, f"Invalid beta parameters: {betas}, expected 3"
        assert all(0.0 <= beta < 1.0 for beta in betas), f"Invalid beta parameters: {betas}"
        if not 0.0 <= weight_decay:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")
        
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        alpha=alpha, T_alpha_beta3=T_alpha_beta3)
        super(AdEMAMix, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdEMAMix, self).__setstate__(state)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            params_with_grad = []
            grads = []
            exp_avgs = []
            exp_avg_sqs = []
            exp_avg_slow = []
            state_steps = []

            for p in group['params']:
                if p.grad is not None:
                    params_with_grad.append(p)
                    if p.grad.is_sparse:
                        raise RuntimeError('AdEMAMix does not support sparse gradients')
                    grads.append(p.grad)

                    state = self.state[p]
                    # Lazy state initialization
                    if len(state) == 0:
                        state['step'] = 0
                        # Exponential moving average of gradient values
                        state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        # Exponential moving average of squared gradient values
                        state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        # Slow exponential moving average
                        state['exp_avg_slow'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                    exp_avgs.append(state['exp_avg'])
                    exp_avg_sqs.append(state['exp_avg_sq'])
                    exp_avg_slow.append(state['exp_avg_slow'])
                    state['step'] += 1
                    state_steps.append(state['step'])

            beta1, beta2, beta3 = group['betas']
            alpha = group['alpha']
            T_alpha_beta3 = group['T_alpha_beta3']

            self._update_adamemix(
                params_with_grad,
                grads,
                exp_avgs,
                exp_avg_sqs,
                exp_avg_slow,
                state_steps,
                beta1=beta1,
                beta2=beta2,
                beta3=beta3,
                alpha=alpha,
                T_alpha_beta3=T_alpha_beta3,
                lr=group['lr'],
                weight_decay=group['weight_decay'],
                eps=group['eps'],
            )

        return loss

    def _update_adamemix(self, params, grads, exp_avgs, exp_avg_sqs, exp_avg_slow, state_steps,
                         beta1, beta2, beta3, alpha, T_alpha_beta3, lr, weight_decay, eps):
        
        for i, param in enumerate(params):
            grad = grads[i]
            exp_avg = exp_avgs[i]
            exp_avg_sq = exp_avg_sqs[i]
            exp_avg_slow_i = exp_avg_slow[i]
            step = state_steps[i]

            bias_correction1 = 1 - beta1 ** step
            bias_correction2 = 1 - beta2 ** step

            if T_alpha_beta3 is not None:
                alpha_t = min(step * alpha / T_alpha_beta3, alpha)
                beta3_t = min(math.exp(math.log(beta1) * math.log(beta3) / 
                              ((1 - step / T_alpha_beta3) * math.log(beta3) + 
                               (step / T_alpha_beta3) * math.log(beta1))), beta3)
            else:
                alpha_t = alpha
                beta3_t = beta3

            # Decay the first and second moment running average coefficient
            exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
            exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
            exp_avg_slow_i.mul_(beta3_t).add_(grad, alpha=1 - beta3_t)

            denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(eps)

            step_size = lr / bias_correction1

            if weight_decay != 0:
                param.add_(param, alpha=-weight_decay * lr)

            param.addcdiv_(exp_avg + alpha_t * exp_avg_slow_i, denom, value=-step_size)


- Run Optimizers on Rosenbrock problem

In [ ]:
# Figure setup
fig, ax = plt.subplots(figsize=(8, 6))

# Parameters
epochs = 500

# Initial condition
initial_position = torch.tensor([-1.0, 1.0], dtype=torch.float32, requires_grad=True).to(device)

# Path colors
colors = {
    "AdamW": "yellow", "Shampoo": "green", "SOAP": "aqua", "Muon": "black", "Adagrad": "red",
    "SGD": "orange", "SGDM": "blue", "AdEMAMix": "violet", "NAG": "grey", "RMSprop": "brown"
}

# Define optimizer parameters
try:
    optim_params = {
        optimizer: initial_position.clone().detach().requires_grad_(True)
        if optimizer != "Muon"
        else initial_position.clone().detach().view(-1, 1).requires_grad_(True)
        for optimizer in colors.keys()
    }
except Exception as e:
    raise RuntimeError(f"Error initializing optimizer parameters: {e}")

# Define optimizers
try:
    optimizers  = {
        "Shampoo": Shampoo([optim_params["Shampoo"].to(device)], lr=0.001),
        "AdamW": optim.AdamW([optim_params["AdamW"].to(device)], lr=0.01, weight_decay=0.01),
        "SOAP": SOAP([optim_params["SOAP"].to(device)], lr=0.01),
        "Muon": Muon([optim_params["Muon"].to(device)], lr=0.01, weight_decay=0.01),
        "AdEMAMix": AdEMAMix([optim_params["AdEMAMix"]], lr=0.01, weight_decay=0.01),
        "RMSprop": optim.RMSprop([optim_params["RMSprop"]], lr=0.01, weight_decay=0.01),
        "Adagrad": optim.Adagrad([optim_params["Adagrad"]], lr=0.01, weight_decay=0.01),
        "SGD": optim.SGD([optim_params["SGD"]], lr=0.001, weight_decay=0.01),
        "SGDM": optim.SGD([optim_params["SGDM"]], lr=0.001, momentum=0.9, weight_decay=0.01),
        "NAG": optim.SGD([optim_params["NAG"]], lr=0.001, momentum=0.9, weight_decay=0.01, nesterov=True),
    }
except Exception as e:
    raise RuntimeError(f"Error initializing optimizers: {e}")

# Create meshgrid for contour plot
x = np.linspace(-2, 3, 400)
y = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x, y)
Z = (1 - X) ** 2 + 100 * (Y - X**2) ** 2  # Rosenbrock function

# Set the contour
contour = ax.contourf(X, Y, Z, levels=np.logspace(0, 5, 20), cmap='coolwarm', alpha=0.6, vmin=-1, vmax=30)
cbar = plt.colorbar(contour, ticks=[0, 10, 1e2, 1e3, 1e4])

def get_positions(epochs: int, optim_params: Dict[str, torch.Tensor]) -> Dict[str, np.ndarray]:
    """
    Run optimization and store the positions of parameters for visualization.
    """
    positions = {key: [param.cpu().clone().detach().numpy()] for key, param in optim_params.items()}
    
    for t in range(epochs):
        for key, optimizer in optimizers.items():
            try:
                optimizer.zero_grad()
                loss = torch_rosenbrock(optim_params[key])  # Compute loss
                loss.backward()
                optimizer.step()
                positions[key].append(optim_params[key].cpu().clone().detach().numpy())
            except Exception as e:
                print(f"Error during optimization step for {key}: {e}")
    
    return {key: np.array(path) for key, path in positions.items()}

def update(frame: int):
    """
    Update function for animation.
    """
    positions = get_positions(epochs, optim_params)
    ax.clear()
    
    # Redraw contour
    contour = ax.contourf(X, Y, Z, levels=np.logspace(0, 5, 20), cmap='coolwarm', alpha=0.6, vmin=-1, vmax=30)
    
    for key, path_pos in positions.items():
        try:
            ax.plot(path_pos[:frame, 0], path_pos[:frame, 1], color=colors[key], label=f"{key} Path", lw=2)
            ax.scatter(*path_pos[frame], color=colors[key], s=100, zorder=5)
            loss_value = torch_rosenbrock(torch.tensor(path_pos[frame])).item()
            ax.text(path_pos[frame, 0], path_pos[frame, 1], f"{loss_value:.2f}", color=colors[key], fontsize=12, ha='center', va='bottom')
        except Exception as e:
            print(f"Error in animation update for {key}: {e}")
    
    ax.set_xlim(-2, 3)
    ax.set_ylim(-1, 3)
    ax.set_title(r"$f(x, y) = (1 - x)^2 + 100(y - x^2)^2$" + f"\nEpoch {frame}", fontsize=12)
    ax.legend(loc="upper left")
    ax.set_xlabel("x")
    ax.set_ylabel("y")


- Generate animation

In [ ]:
def animate_and_save_figure(
    fig: plt.Figure,
    update: Callable[[int], None],
    epochs: int,
    interval: int = 100,
    repeat: bool = False,
    title: str = "animation",
    fps: int = 15
) -> None:
    """
    Creates an animation and saves it as a GIF.
    
    Parameters:
        fig (plt.Figure): The figure to animate.
        update (Callable[[int], None]): The update function for animation.
        epochs (int): Number of frames (epochs) for the animation.
        interval (int, optional): Interval between frames in milliseconds (default: 10ms).
        repeat (bool, optional): Whether the animation should loop (default: False).
        title (str, optional): Name of the output GIF file (default: "animation").
        fps (int, optional): Frames per second for the saved GIF (default: 15).
    
    Returns:
        None
    """
    try:
        ani = FuncAnimation(fig, update, frames=epochs, interval=interval, repeat=repeat)
        ani.save(f"{title}.gif", writer="pillow", fps=fps)
        print(f"Animation saved as {title}.gif")
    except Exception as e:
        raise RuntimeError(f"Error in animation process: {e}")

In [ ]:
animate_and_save_figure(fig,update,20, 100)